In [3]:
using Plots, Distributions
include("./src/supportFunctions.ji")
include("./src/qgplsim.ji")
pwd()

"/Users/kyan/Projects/qgplsim"

In [35]:
t1 = TDist(1)
t3 = TDist(3)


function eps(n, etype)
    
    if etype == 1
            e = randn(n)/2
        elseif etype == 2
            e = rand(t3, n) * sqrt(3) / 10    
        elseif etype == 3
            e = rand(t1, n) / 10
        else
            e = randn(n)
            e = (e.^4 .- 3) .* 0.3             
    end
    e

end

eps (generic function with 1 method)

# Case 1
$y = X \alpha + exp(X \theta) + \epsilon$

In [36]:
function simu_data1(X, α, θ, ϵ)

    y = X*α + exp.(X*θ) + ϵ

end

simu_data1 (generic function with 1 method)

## test 

In [189]:
∑ = zeros(5,5)

for i in 1:5
    for j in i:5
        ∑[i, j] = 0.5^(j - i)
        ∑[j, i] = ∑[i, j]
    end
end

∑ = sqrt.(∑)

n = 200
alpha = [0, 1, 0, -2, 0]
theta = [2, 0, -1, 0, 2]/3

U = randn(n, 5)
X = U * ∑
e = eps(n, 1)
y = simu_data1(X, alpha, theta, e)
Z = zeros(n, 1) # rand(n, 1) .> 0.5

tau = [0.5]
widthExp = -0.17
model1 = qgplsim.model(X, Z, y, tau, widthExp)
qgplsim.estimator(model1, "optim_qr")
qgplsim.print_model(model1)

alpha => [0.0793, 0.9967, 0.1066, -2.1169, 0.1533]
theta => [0.5964, 0.0535, -0.4542, 0.0722, 0.6558]
gamma => [0.0]
beta => [0.0]


## Monte Carlo Simulation

In [166]:
using Base.Threads
nth = Threads.nthreads()
println("threads for parallel computing:  ", nth)

threads for parallel computing:  4


In [167]:
n = 200
alpha = [0, 1, 0, -2, 0]
theta = [2, 0, -1, 0, 2]/3
Z = zeros(n, 1)
n_repeat = 25
n_repeat * nth

100

### $\epsilon$ ~ $N(0, 1)/2$

In [168]:
Alpha1 = zeros(n_repeat * nth, 5)
Theta1 = zeros(n_repeat * nth, 5)
@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = rand(n) / 2
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha1[k, :] = modelk.alpha
        Theta1[k, :] = modelk.theta
    end
end



In [169]:
EE1 = sqrt.(1 .- abs.(Theta1 * theta)) 
println(mean(EE1), " ", std(EE1))

0.13994777542057338 0.158703032115617


In [170]:
mean(Theta1, dims = 1)

1×5 Matrix{Float64}:
 0.658525  -0.0069852  -0.333576  0.0224074  0.600136

### $\epsilon$ ~ $\sqrt 3 t(3) / 10$

In [172]:
Alpha2 = zeros(n_repeat * nth, 5)
Theta2 = zeros(n_repeat * nth, 5)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = rand(t3, n) * sqrt(3) / 10
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha2[k, :] = modelk.alpha
        Theta2[k, :] = modelk.theta
    end
end

In [173]:
mean(Theta2, dims = 1)

1×5 Matrix{Float64}:
 0.618235  0.0223955  -0.331152  0.0309383  0.577526

In [174]:
EE2 = sqrt.(1 .- abs.(Theta2 * theta)) 
println(mean(EE2), " ", std(EE2))

0.1773731420044237 0.21078261894154507


### $\epsilon$ ~ $t(1)/10$

In [176]:
Alpha3 = zeros(n_repeat * nth, 5)
Theta3 = zeros(n_repeat * nth, 5)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = rand(t1, n) / 10
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha3[k, :] = modelk.alpha
        Theta3[k, :] = modelk.theta
    end
end

In [177]:
mean(Theta3, dims = 1)

1×5 Matrix{Float64}:
 0.647159  0.00469007  -0.344865  0.0617279  0.600015

In [178]:
EE3 = sqrt.(1 .- abs.(Theta3 * theta)) 
println(mean(EE3), " ", std(EE3))

0.1572008337416937 0.15656874707896248


### $\epsilon$ ~ $0.3(N(0, 1)^4 - 3)$

In [179]:
Alpha4 = zeros(n_repeat * nth, 5)
Theta4 = zeros(n_repeat * nth, 5)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = randn(n)
        e = (e.^4 .- 3) .* 0.3
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha4[k, :] = modelk.alpha
        Theta4[k, :] = modelk.theta
    end
end

In [180]:
mean(Theta4, dims = 1)

1×5 Matrix{Float64}:
 0.631767  0.0122769  -0.346298  0.0687969  0.577738

In [181]:
EE4 = sqrt.(1 .- abs.(Theta4 * theta)) 
println(mean(EE4), " ", std(EE4))

0.21594105838062394 0.1751448733458884


# Case 2
$y = X \alpha + Z \beta + exp(X \theta + Z \gamma) + \epsilon$

In [568]:
function simu_data2(X, α, θ, Z, β, γ, ϵ)

    y = X*α  + Z*β + exp.(X*θ + Z*γ) + ϵ

end

function xyz2(n, alpha, theta, beta, gamma, e)
    ∑ = zeros(5,5)

    for i in 1:5
        for j in i:5
            ∑[i, j] = 0.5^(j - i)
            ∑[j, i] = ∑[i, j]
        end
    end

    ∑ = sqrt.(∑)


    U = randn(n, 5)
    X = U * ∑

    Z = zeros(n, 2)
    Z[:, 1] = rand(n, 1) .> 0.25
    Z[:, 2] = rand(n, 1) .> 0.5

    y = simu_data2(X, alpha, theta, Z, beta, gamma, e)
    X, y, Z
    
end

xyz2 (generic function with 1 method)

## test 

In [282]:
n = 400
alpha = [0, 1, 0, -2, 0]
theta = [2, 0, -1, 0, 2]/3
beta = [1.0, 1.0]
gamma = [1.5, -1.5] 
e = rand(n) / 2

X, y, Z = xyz2(n, alpha, theta, beta, gamma, e)

tau = [0.5]
widthExp = -0.17
model1 = qgplsim.model(X, Z, y, tau, widthExp)
qgplsim.estimator(model1, "optim_qr")
qgplsim.print_model(model1)

alpha => [0.1997, 1.0646, 0.0589, -2.1028, 0.0817]
theta => [0.5978, 0.0114, -0.4006, 0.0782, 0.6899]
gamma => [0.5324, -0.5605]
beta => [13.056, -4.915]


In [278]:
model1.widthExp

-0.17

## Monte Carlo Simulation

In [ ]:
using Base.Threads
nth = Threads.nthreads()
println("threads for parallel computing:  ", nth)

rc_mg = []

In [516]:
n = 800
alpha = [0, 1, 0, -2, 0]
theta = [2, 0, -1, 0, 2]/3
beta = [1, 0]
gamma = [0.0, -1.0]./3 * 3 
n_repeat = 25
n_repeat * nth

100

### $\epsilon$ ~ $N(0, 1)/2$

In [517]:
Alpha1 = zeros(n_repeat * nth, 5)
Theta1 = zeros(n_repeat * nth, 5)
Gamma1 = zeros(n_repeat * nth, 2)


@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        e = randn(n) / 2
        X, y, Z = xyz2(n, alpha, theta, beta, gamma, e)

        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha1[k, :] = modelk.alpha
        Theta1[k, :] = modelk.theta
        Gamma1[k, :] = modelk.gamma

    end
end



In [518]:
EE1 = sqrt.(1 .- abs.(Theta1 * theta)) 
println(mean(EE1), " ", std(EE1))

0.14172182717484455 0.1660180877623868


In [519]:
mean(Alpha1, dims = 1)

1×5 Matrix{Float64}:
 0.0336637  0.918952  -0.00441354  -1.77664  0.0267027

In [520]:
mean(Theta1, dims = 1)

1×5 Matrix{Float64}:
 0.632019  -0.0103497  -0.345464  0.0425938  0.607472

In [521]:
mg = mean(Gamma1, dims = 1)

1×2 Matrix{Float64}:
 -0.0103822  -0.358116

In [522]:
push!(rc_mg, mg)

5-element Vector{Any}:
 [0.004859570197597576 -0.04845031202682143]
 [0.00479407781990095 -0.08473400639663248]
 [-0.0033128801551007923 -0.18123445475148298]
 [0.0019663682468473056 -0.27076572576931485]
 [-0.010382243273092052 -0.3581158922391516]

In [523]:
rc_mg

5-element Vector{Any}:
 [0.004859570197597576 -0.04845031202682143]
 [0.00479407781990095 -0.08473400639663248]
 [-0.0033128801551007923 -0.18123445475148298]
 [0.0019663682468473056 -0.27076572576931485]
 [-0.010382243273092052 -0.3581158922391516]

In [532]:
norm.(rc_mg11)

5-element Vector{Float64}:
 0.06386626773013654
 0.13430869844029475
 0.2542784243719414
 0.39064902123508083
 0.5424844464753418

In [531]:
rc_mg11 

5-element Vector{Any}:
 [0.04792928972782212 -0.04220999099696617]
 [0.09806324865875911 -0.0917737747900587]
 [0.1797354069161139 -0.179868564796009]
 [0.2730871018515566 -0.2793386700660046]
 [0.3758228653703359 -0.391211641611658]

### $\epsilon$ ~ $\sqrt 3 t(3) / 10$

In [150]:
Alpha2 = zeros(n_repeat * nth, 5)
Theta2 = zeros(n_repeat * nth, 5)
Gamma2 = zeros(n_repeat * nth, 2)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        e = rand(t3, n) * sqrt(3) / 10
        X, y, Z = xyz2(n, alpha, theta, beta, gamma, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha2[k, :] = modelk.alpha
        Theta2[k, :] = modelk.theta
        Gamma2[k, :] = modelk.gamma

        
    end
end

In [151]:
mean(Theta2, dims = 1)

1×5 Matrix{Float64}:
 0.6453  0.00128617  -0.32176  0.0106369  0.572918

In [152]:
EE2 = sqrt.(1 .- abs.(Theta2 * theta)) 
println(mean(EE2), " ", std(EE2))

0.17522982401106535 0.21081039981797559


In [230]:
EEa = sqrt.(abs.(5 .- abs.(Alpha2 * alpha))/5)
println(mean(EEa), " ", std(EEa))

0.2604622974978035 0.2441158621826322


In [153]:
mean(Gamma2, dims = 1)

1×2 Matrix{Float64}:
 -0.0015759  -0.36258

### $\epsilon$ ~ $t(1)/10$

In [154]:
Alpha3 = zeros(n_repeat * nth, 5)
Theta3 = zeros(n_repeat * nth, 5)
Gamma3 = zeros(n_repeat * nth, 2)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        e = rand(t1, n) / 10
        X, y, Z = xyz(n, alpha, theta, beta, gamma, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha3[k, :] = modelk.alpha
        Theta3[k, :] = modelk.theta
        Gamma3[k, :] = modelk.gamma

        
    end
end

In [155]:
mean(Theta3, dims = 1)

1×5 Matrix{Float64}:
 0.635217  -0.0285129  -0.33521  0.0350691  0.598973

In [264]:
EE3 = sqrt.(1 .- abs.(Theta3 * theta)) 
println(mean(EE3), " ", std(EE3))

0.1572008337416937 0.15656874707896248


In [266]:
na3 = norm.([Alpha3[k, :] for k in 1:100] )
EEa = sqrt.(abs.(1 .- abs.(Alpha3 ./ na3 * alpha / norm(alpha))))
println(mean(EEa), " ", std(EEa))

0.10734724786839749 0.10430637727052751


In [157]:
mean(Gamma3, dims = 1)

1×2 Matrix{Float64}:
 -0.00624061  -0.363546

### $\epsilon$ ~ $0.3(N(0, 1)^4 - 3)$

In [158]:
Alpha4 = zeros(n_repeat * nth, 5)
Theta4 = zeros(n_repeat * nth, 5)
Gamma4 = zeros(n_repeat * nth, 2)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        e = randn(n)
        e = (e.^4 .- 3) .* 0.3
        X, y, Z = xyz(n, alpha, theta, beta, gamma, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha4[k, :] = modelk.alpha
        Theta4[k, :] = modelk.theta
        Gamma4[k, :] = modelk.gamma
        
    end
end

In [159]:
mean(Theta4, dims = 1)

1×5 Matrix{Float64}:
 0.623733  -0.0435303  -0.349124  0.0472179  0.553621

In [160]:
EE4 = sqrt.(1 .- abs.(Theta4 * theta)) 
println(mean(EE4), " ", std(EE4))

0.23493551471318813 0.20054916315879562


In [261]:
na4 = norm.([Alpha4[k, :] for k in 1:100] )
EEa = sqrt.(abs.(1 .- Alpha4./na4 * alpha / norm(alpha)))
println(mean(EEa), " ", std(EEa))

In [161]:
mean(Gamma4, dims = 1)

1×2 Matrix{Float64}:
 -0.0045003  -0.321222

# Case 3
$y = X \alpha + Z \beta + log( 1 + exp(X \theta + Z \gamma - 1)) + 0.2\epsilon$

In [13]:
n1 = Normal(0, 1)
g(x) = cdf(n1, x)

function simu_data3(X, α, θ, Z, β, γ, ϵ)

#     y = X*α  + Z*β + log.( 1 .+ exp.(X*θ + Z*γ .- 1)) + ϵ
    y = X*α  + Z*β + g.((X*θ + Z*γ .- 1).^2) + ϵ


end

function xyz3(n, alpha, theta, beta, gamma, e)

    U = randn(n, 2)
    X = U

    Z = zeros(n, 2)
    Z[:, 1] = rand(n, 1) .> 0.5
    r2 = rand(n, 1)
    Z[:, 2] = (r2 .> 0.25) + (r2 .> 0.75)
    # Z = Z / 4 * 3
    y = simu_data3(X, alpha, theta, Z, beta, gamma, e)
    X, y, Z
    
end

xyz3 (generic function with 1 method)

## test 

In [153]:
include("./src/qgplsim.ji")

n = 500
alpha = [1, -2]
theta = [2, 1]/sqrt(5)
beta = [1.0, -1.0]/4 * 4
gamma = [1.0, 1.0]/8 * 4

e = eps(n, 2)

X, y, Z = xyz3(n, alpha, theta, beta, gamma, 0.2e)

tau = [0.5]
widthExp = -0.17
model1 = qgplsim.model(X, Z, y, tau, widthExp)
qgplsim.estimator(model1, "optim_qr")
qgplsim.print_model(model1)

alpha => [0.9934, -1.9993]
theta => [0.8955, 0.445]
gamma => [0.5384, 0.6111]
beta => [0.9142, -1.0946]


## Monte Carlo Simulation

In [51]:
using Base.Threads
nth = Threads.nthreads()
n_repeat = 34
n_repeat * nth

102

In [120]:
n = 500
alpha = [1, -2]
theta = [2, 1]/sqrt(5)
beta = [1.0, -1.0] / 2
gamma = [1.0, 1.0]/4 * 2
tau = [0.5]
widthExp = -0.27

n_repeat = 34
n_repeat * nth

102

In [121]:
Alpha1 = zeros(n_repeat * nth, 2)
Theta1 = zeros(n_repeat * nth, 2)
Gamma1 = zeros(n_repeat * nth, 2)
Beta1 = zeros(n_repeat * nth, 2)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        e = eps(n, 2)
        X, y, Z = xyz3(n, alpha, theta, beta, gamma, e)

        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha1[k, :] = modelk.alpha
        Theta1[k, :] = modelk.theta
        Gamma1[k, :] = modelk.gamma
        Beta1[k, :] = modelk.beta

    end
end



┌ Warning: The clustering cost increased at iteration #13
└ @ Clustering /Users/kyan/.julia/packages/Clustering/tt9vc/src/kmeans.jl:188
┌ Warning: The clustering cost increased at iteration #21
└ @ Clustering /Users/kyan/.julia/packages/Clustering/tt9vc/src/kmeans.jl:188


In [122]:
EE1 = sqrt.(1 .- abs.(Theta1 * theta)) 
println(mean(EE1), " ", std(EE1))

0.3171413978665687 0.3590843341002056


In [123]:
ma = mean(Alpha1, dims = 1)

1×2 Matrix{Float64}:
 0.717948  -1.3441

In [124]:
mt = mean(Theta1, dims = 1)

1×2 Matrix{Float64}:
 0.737065  0.120008

In [125]:
mg = mean(Gamma1, dims = 1)

1×2 Matrix{Float64}:
 0.186977  0.250028

In [126]:
mb = mean(Beta1, dims = 1)

1×2 Matrix{Float64}:
 0.459464  -0.538871

# Tabular

In [ ]:
N = [500, 1000]
Etype = [1,2,3,4]

len_n = length(N)
len_e = length(Etype)

meanEE_theta = zeros(len_n, len_e)
stdEE_theta = zeros(len_n, len_e)

meanEE_alpha = zeros(len_n, len_e)
stdEE_alpha = zeros(len_n, len_e)
n_repeat = 100
tau = [0.5]
widthExp = -0.17

alpha = [0, 1, 0, -2, 0]
theta = [2, 0, -1, 0, 2]/3
beta = [0, 0]
gamma = [0, -0] 

p = length(alpha)
q = length(beta)

@threads for j in 1:len_e
    
    etype = Etype[j]    
    
    for i in 1:len_n  

        n = N[i] 
       
        Alpha = zeros(n_repeat, p)
        Theta = zeros(n_repeat, p)
        Beta = zeros(n_repeat, q)
        Gamma = zeros(n_repeat, q)
        
        for k in 1:n_repeat
            
            e = eps(n, etype)
            X, y, Z = xyz(n, alpha, theta, beta, gamma, e)
            Z = zeros(n, 1)
            modelk = qgplsim.model(X, Z, y, tau, widthExp)
            qgplsim.estimator(modelk, "optim_qr")
            Alpha[k, :] = modelk.alpha
            Theta[k, :] = modelk.theta
            
        end
        EE = sqrt.(1 .- abs.(Theta * theta)) 
        meanEE_theta[i,j] = mean(EE)
        stdEE_theta[i,j] = std(EE)
        norm_Alpha = norm.([Alpha[k, :] for k in 1:n_repeat]
        EE = sqrt.(abs.(1 .- abs.(Alpha ./ norm_Alpha * alpha / norm(alpha)))) 
        meanEE_alpha[i,j] = mean(EE)
        stdEE_alpha[i,j] = std(EE)
    end
end

In [241]:
table_theta = tabular(meanEE_theta, stdEE_theta)

4×4 Matrix{String}:
 "0.5865(.2482)"  "0.5490(.2372)"  "0.5874(.2378)"  "0.6538(.2138)"
 "0.4125(.2088)"  "0.2299(.1540)"  "0.3585(.2217)"  "0.4950(.2373)"
 "0.1943(.1407)"  "0.1527(.1721)"  "0.1563(.1349)"  "0.2239(.1737)"
 "0.1359(.1712)"  "0.0892(.1054)"  "0.1324(.1811)"  "0.1554(.1724)"

In [244]:
table_alpha = tabular(meanEE_alpha, stdEE_alpha)

4×4 Matrix{String}:
 "0.7268(.1465)"  "0.7387(.1491)"  "0.7136(.1547)"  "0.7283(.1570)"
 "0.5548(.1811)"  "0.3589(.1506)"  "0.4800(.1817)"  "0.5565(.1712)"
 "0.2668(.1145)"  "0.1996(.1255)"  "0.2314(.1094)"  "0.3137(.1667)"
 "0.2123(.1307)"  "0.1364(.0678)"  "0.1604(.1285)"  "0.2116(.1350)"

In [245]:
using DataFrames, CSV
df = DataFrame(table_alpha)
CSV.write("table_alpha.csv", df)

"tablez_alpha.csv"